# 2021/10/18
# 本代码使用merra2资料来计算地面2m风的气候态
# 时间跨度1985-2005

In [1]:
import os
import xarray as xr
import numpy as np
from cdo import *
cdo = Cdo()

module_path = ["/home/sun/mycode/module/","/data5/2019swh/mycode/module/"]
sys.path.append(module_path[0])
from module_sun import *

In [2]:
path1  =  "/home/sun/mydown/merra2_u2v2/"
path2  =  "/home/sun/data/merra2_u2v2_daymean/"

list1  =  os.listdir(path1) ; list1.sort()
f0     =  xr.open_dataset(path1+list1[1])
#声明最终结果数组
u2_climate  =  np.zeros((365,f0.U2M.data.shape[1],f0.U2M.data.shape[2]))
v2_climate  =  u2_climate.copy()

start  =  1985 ; end  =  1999 # 1999年有部分数据出错了



In [3]:
for yyyy in range(start,end):
    list_year  =  []
    for dddd in range(1,366):
        date1  =  str(yyyy)+out_date(1981,dddd)
        for name1 in list1:
            if date1 in name1:
                list_year.append(name1)
    for dddd in range(0,len(list_year)):
        cdo.daymean(input=path1+list_year[dddd],output=path2+list_year[dddd])
        f1  =  xr.open_dataset(path2+list_year[dddd])
        u2  =  f1.U2M.data[0,:,:]
        v2  =  f1.V2M.data[0,:,:]

        u2_climate[dddd,:] += u2/(end-start)
        v2_climate[dddd,:] += v2/(end-start)

In [5]:
ncfile  =  xr.Dataset(
    {
        "u2_climate": (["time", "lat", "lon"], u2_climate),
        "v2_climate": (["time", "lat", "lon"], v2_climate),
    },
    coords={
        "lon": (["lon"], f0.lon.data),
        "lat": (["lat"], f0.lat.data),
        "time": (["time"], np.linspace(1,365,365)),
    },
)
ncfile.u2_climate.attrs  =  f0.U2M.attrs
ncfile.v2_climate.attrs  =  f0.V2M.attrs
ncfile["lat"].attrs  =  f0["lat"].attrs
ncfile["lon"].attrs  =  f0["lon"].attrs
ncfile.to_netcdf("/home/sun/data/merra2_u2v2_climate.nc")